<a href="https://colab.research.google.com/github/pranjalipilankar/Moon-Map/blob/main/train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#uncomment for colab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/Moon\ Mapping/Models/Moon-Mapping/AI\ Models/SwinIR/
%ls

/content/drive/.shortcut-targets-by-id/19TyNbSyd7i1igZMVw4YRX5xonl55yZTb/Moon Mapping/Models/Moon-Mapping/AI Models/SwinIR
 architecture.gdoc                                         lightning_logs/
 architecture.txt                                          main_test_swinir.py
 check.png                                                 mismatch.txt
 Checkpoints/                                              models/
 cog.yaml                                                  model_zoo/
 DataSet/                                                 'Moon mapping.pdf'
 DataSet_Gray_8x/                                          network_swinir_state_dict.txt
 DataSet_Gray_High/                                        predict.py
'DataSet_ Grayscale'/                                      README.md
 download-weights.sh                                       testsets/
 experiments/                                              train/
 Extracting-Intersection-Coordinates-Between-Images.pptx   train2.ipynb
 f

In [ ]:
%pip install timm

In [ ]:
import torch
from models.network_swinir import SwinIR as net
from utils import util_calculate_psnr_ssim as util
from torchvision import transforms
from PIL import Image


In [ ]:
model = net(upscale=1, in_chans=1, img_size=128, window_size=8,
            img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
            mlp_ratio=2, upsampler='', resi_connection='1conv')
param_key_g = 'params'

model_path = r'/content/drive/MyDrive/Moon Mapping/Models/Moon-Mapping/AI Models/SwinIR/experiments/pretrained_models/004_grayDN_DFWB_s128w8_SwinIR-M_noise50.pth'
pretrained_model = torch.load(model_path)
model.load_state_dict(pretrained_model[param_key_g] if param_key_g in pretrained_model.keys() else pretrained_model, strict=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device=device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SwinIR(
  (conv_first): Conv2d(1, 180, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (patch_embed): PatchEmbed(
    (norm): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
  )
  (patch_unembed): PatchUnEmbed()
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): RSTB(
      (residual_group): BasicLayer(
        dim=180, input_resolution=(128, 128), depth=6
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            dim=180, input_resolution=(128, 128), num_heads=6, window_size=8, shift_size=0, mlp_ratio=2
            (norm1): LayerNorm((180,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              dim=180, window_size=(8, 8), num_heads=6
              (qkv): Linear(in_features=180, out_features=540, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=180, out_features=180, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
   

In [ ]:
from torchsummary import summary
summary(model, (1, 128, 128))

In [ ]:
# container = list(model.children())
# # container[7] = torch.nn.ReLU()
# model = torch.nn.Sequential(*container)
# list(model.children())

In [ ]:
img = Image.open(r'/content/drive/MyDrive/Moon Mapping/Models/Moon-Mapping/AI Models/SwinIR/DataSet/GT/324.png')
img = img.resize((128, 128))
img.show()
img = transforms.PILToTensor()(img).to(device=device)[0,:,:]
img = img.unsqueeze(0).unsqueeze(0).to(dtype=torch.float32)
print(img.size())
model.eval()
img = model(img).to(device='cpu')
img = img.squeeze(0)
print(img.size())
img = transforms.ToPILImage()(img).show()

torch.Size([1, 1, 128, 128])
torch.Size([1, 128, 128])


In [ ]:
import torch
ckpt = torch.load('Checkpoints/model_1_pl_perceptual_loss/epoch=18-train_loss=1.14.ckpt')
print(ckpt)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
import pytorch_lightning as pl
from models.network_swinir import default_model_import
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR
# %pip install torch-lr-finder
from torch_lr_finder import LRFinder
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import os
# %pip install pytorch-msssim
from pytorch_msssim import ssim
from models.network_swinir import SwinIR as net

/home/sukhvansh/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class PerceptualLoss(nn.Module):
    def __init__(self, resize=True):
        super(PerceptualLoss, self).__init__()
        # Use pre-trained VGG model (features only)
        blocks = []
        blocks.append(models.vgg16(pretrained=True).features[:4].eval())
        blocks.append(models.vgg16(pretrained=True).features[4:9].eval())
        self.blocks = torch.nn.ModuleList(blocks)

        # Freeze weights
        for block in self.blocks:
            for p in block.parameters():
                p.requires_grad = False

        self.transform = nn.functional.interpolate.Bilinear2d(scale_factor=1.0) if resize else None
        self.register_buffer("mean", torch.tensor([0.485]).view(1, 1, 1, 1))
        self.register_buffer("std", torch.tensor([0.229]).view(1, 1, 1, 1))

    def forward(self, input, target, feature_weights=[1.0, 1.0]):
        """
        Calculates perceptual loss between input and target grayscale images.

        Args:
            input: Tensor of generated images (N, 1, 256, 256).
            target: Tensor of ground truth images (N, 1, 256, 256).
            feature_weights: List of weights for each feature layer (default: [1.0, 1.0]).

        Returns:
            Total perceptual loss (scalar).
        """

        if input.size(1) != 1:
            raise ValueError("Input must be a grayscale image (channel = 1)")
        if target.size(1) != 1:
            raise ValueError("Target must be a grayscale image (channel = 1)")

        # Preprocess input and target
        input = input.repeat(1, 3, 1, 1)  # Replicate grayscale to 3 channels
        target = target.repeat(1, 3, 1, 1)
        input = (input - self.mean) / self.std
        target = (target - self.mean) / self.std

        if self.transform:
            input = self.transform(input)
            target = self.transform(target)

        # Extract features from both images
        f_input = []
        f_target = []
        for block in self.blocks:
            input = block(input)
            target = block(target)
            f_input.append(input)
            f_target.append(target)

        # Calculate loss for each feature layer
        loss = 0.0
        for i, (f_in, f_out) in enumerate(zip(f_input, f_target)):
            loss += feature_weights[i] * torch.nn.functional.l1_loss(f_in, f_out)

        return loss

class CombinedLoss(nn.Module):
  def __init__(self, perceptual_weight=1.0, mse_weight=1.0, resize=False):
    super(CombinedLoss, self).__init__()
    self.perceptual_loss = PerceptualLoss(resize=resize)
    self.mse_loss = nn.MSELoss()
    self.perceptual_weight = perceptual_weight
    self.mse_weight = mse_weight

  def forward(self, input, target):
    # Calculate perceptual and MSE loss
    perceptual_loss = self.perceptual_loss(input, target)
    mse_loss = self.mse_weight * self.mse_loss(input, target)

    # Combine losses with weights
    loss = self.perceptual_weight * perceptual_loss + mse_loss
    return loss

class model1(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = net(upscale=4, in_chans=1, img_size=64, window_size=8,
                        img_range=1., depths=[6, 6, 6, 6, 6, 6], embed_dim=180, num_heads=[6, 6, 6, 6, 6, 6],
                        mlp_ratio=2, upsampler='nearest+conv', resi_connection='1conv')
        self.criterion = CombinedLoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        # access your optimizers with use_pl_optimizer=False. Default is True
        inputs, labels = batch[0], batch[1]
        outputs = self.model(inputs)
        loss = self.criterion(outputs, labels)
        self.log('train_loss', loss)
        return loss


In [ ]:
model = model1.load_from_checkpoint('Checkpoints/model_1_pl_perceptual_loss/epoch=26-train_loss=1.03.ckpt')

In [ ]:
img = Image.open('DataSet_Gray_High/Train/324.png')
img.show()
img = transforms.PILToTensor()(img).to(device='cuda')[0,:,:]
img = img.unsqueeze(0).unsqueeze(0).to(dtype=torch.float32)
print(img.size())
model.eval()
img = model.forward(img).to(device='cpu')
img = img.squeeze(0)
print(img.size())
img = transforms.ToPILImage()(img).save('check.png')

torch.Size([1, 1, 64, 64])
torch.Size([1, 256, 256])
